In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from torchvision import transforms
import torch.utils.data
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import string
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mr.J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mr.J\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mr.J\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
RANDOM_SEED  = 123
torch.manual_seed(RANDOM_SEED)

max_sequence    = 100

In [5]:
df = pd.read_csv('dataset/AMAZON_FASHION_5.csv')
df = df.dropna(subset=['reviewText'])
df = df.sample(frac=1, random_state=1)

def remove_punctuation(text):
    if isinstance(text, float):
        print(text)
    return text.translate(str.maketrans('', '', string.punctuation)).lower()
def remove_stop_words(tokens):
    return [token for token in tokens if token not in stop_words]
def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
df.head()

,overall,verified,reviewTime,reviewerID,asin,style/Size:,style/Color:,reviewerName,reviewText,summary,unixReviewTime,style/Size Name:,style/Style:,vote,image/0,image/1,image/2
1475,3,True,"01 31, 2017",AODFGQL9CC7G7,B009MA34NY,7.5 B(M) US,Racer Blue/Obsidian/Blue Tint,ZS87,Squeaky,Three Stars,1485820800,NaN,NaN,3.0,NaN,NaN,NaN
483,3,True,"06 21, 2016",A2QJWHL4HY9FMF,B001IKJOLW,10 B(M) US,Black/Wolf Grey/White/Pink,Amazon Customer,"Super cute shoe however runs wide and big, I'm...",Cute,1466467200,NaN,NaN,2.0,NaN,NaN,NaN
1941,5,True,"06 10, 2016",AIP4C9MG13COB,B0092UF54A,10.5 B(M) US,Black/White/Anthracite/Stealth,Cheryl A Stoneham,Comfortable and lightweight. Need a good train...,Comfortable and lightweight,1465516800,NaN,NaN,10.0,NaN,NaN,NaN
682,5,False,"05 19, 2017",AT7UEQ7UJB0RU,B0058YEJ5K,10 B(M) US,Cool Grey/Pure Platinum/White/Volt,J.T.,Love them absolutely comfortable. I'm very pic...,Amazing training Sneakers.,1495152000,NaN,NaN,4.0,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...
734,5,True,"03 8, 2017",A1LLDHUO4OLXYI,B0058YEJ5K,10 B(M) US,Black/White/Anthracite/Stealth,mommyof2,Super comfy!!,Five Stars,1488931200,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
review = df['reviewText'].apply(remove_punctuation).apply(word_tokenize).apply(remove_stop_words).apply(lemmatize_words)

In [7]:
def map_label(sentiment):
    return sentiment-1

sentiment = df['overall'].apply(map_label)

In [8]:
X = review.to_list()
y = sentiment[review.index].to_numpy()
print(len(X))
print(len(y))

3160
3160


In [9]:
all_words = [token for phrase in X  for token in phrase ]
vocab = Counter(all_words)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
encoded_review = [[vocab_to_int[token] for token in phrase] for phrase in X]



features = np.zeros((len(encoded_review), max_sequence), dtype=np.int32)
for i, e in enumerate(encoded_review):
    e_len = len(e)
    if e_len <= max_sequence:
        zeros   = list(np.zeros(max_sequence-e_len))
        new     = zeros + e
    else:
        new     = e[:max_sequence]
    features[i,:] = np.array(new)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(features, y, test_size=.2, random_state=42)

In [11]:
class imdbdataset(Dataset):
    def __init__(self, x, y, transform=None) -> None:
        super(imdbdataset, self).__init__()
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    # def get_labels(self):   return self.y

class ToTensor:
    def __call__(self, sample):
        x, y = sample
        x = np.array(x) if not isinstance(x, np.ndarray) else x
        y = np.array(y) if not isinstance(y, np.ndarray) else y
        return torch.from_numpy(x), torch.from_numpy(y).long()

compose = transforms.Compose([
    ToTensor(),
])

train_data_set = imdbdataset(x_train, y_train, transform=compose)
test_data_set  = imdbdataset(x_test,  y_test,  transform=compose)



In [12]:
train_loader  = DataLoader(train_data_set, batch_size=16, shuffle=True)
test_loader   = DataLoader(test_data_set,  batch_size=16, shuffle=False)


In [13]:
example = iter(train_loader)
feature, label = next(example)
print(feature.shape, label)

torch.Size([16, 100]) tensor([4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 1, 2, 4, 3])


In [14]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, rnn_type='simple'):
        super(RNN, self).__init__()
        self.embedding  = nn.Embedding(input_dim, embedding_dim)

        self.rnn_type = rnn_type

        if rnn_type == 'gru':
            self.rnn        = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == 'lstm':
            self.rnn        = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        else:
            self.rnn        = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        self.fc         = nn.Linear(hidden_dim, output_dim)

    def forward(self, tt):
        embedded = self.embedding(tt)

        out, h = self.rnn(embedded)

        if self.rnn_type == 'lstm':
            h = h[0]

        output = self.fc(h.squeeze_(0))
        return output

learning_rate   = .001
num_epochs      = 3
embedding_dim   = 100
hidden_dim      = 128
num_classes     = 5


model = RNN(input_dim=len(vocab_to_int)+1, embedding_dim=embedding_dim,
            hidden_dim=hidden_dim, output_dim=num_classes, rnn_type='lstm').to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
def test(class_test=False):
    test_loss   = 0.0
    total       = 0
    correct     = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    with torch.no_grad():
        for text, label in test_loader:
            text, label = text.to(device), label.to(device)
            output = model(text)
            loss = criterion(output, label)
            test_loss += loss.item() * text.size(0)

            _, predicted = torch.max(output, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            for i in range(len(label)):
                label_ = label[i]
                pred  = predicted[i]
                if (label_ == pred):
                    n_class_correct[label_] += 1
                n_class_samples[label_] += 1

    test_loss /= len(test_loader.dataset)
    accuracy = correct / total
    print(f" Test Loss: {test_loss}, Test Accuracy: {accuracy}")
    if class_test:
        for i in range(num_classes):
            acc = 100 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {i}:{acc}')


In [16]:
for epoch in range(num_epochs):
    train_loss = 0.0
    for text, label in train_loader:
        text, label = text.to(device), label.to(device)

        output = model(text)

        l = criterion(output, label)

        optimizer.zero_grad()

        l.backward()

        optimizer.step()

        train_loss += l.item()
    train_loss /= len(train_loader.dataset)
    print(f"epoch {epoch+1}/{num_epochs} loss: {train_loss}", end='')
    test()
test(class_test=True)

epoch 1/3 loss: 0.0537180266802824 Test Loss: 0.5189613858355752, Test Accuracy: 0.8306962025316456
epoch 2/3 loss: 0.01656917480551461 Test Loss: 0.18245546538618546, Test Accuracy: 0.9509493670886076
epoch 3/3 loss: 0.0045908226434854624 Test Loss: 0.1157798509620413, Test Accuracy: 0.9762658227848101
 Test Loss: 0.1157798509620413, Test Accuracy: 0.9762658227848101
Accuracy of 0:100.0
Accuracy of 1:100.0
Accuracy of 2:89.70588235294117
Accuracy of 3:94.25287356321839
Accuracy of 4:99.29411764705883
